In [1]:
import json
import logging
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn import preprocessing
import sys
from environs import Env
import torch
import detectron2

In [2]:
detectron2.__version__

'0.1.3'

In [2]:
sys.path.append('../')

env = Env()
env.read_env()

# Get training dataframe
path_data = Path(env("path_raw"))
path_image = path_data / "train/"
path_data_interim = Path(env("path_interim"))

In [3]:
from imaterialist.data.datasets.make_dataset import load_dataset_into_dataframes, create_datadict, attr_str_to_list                  

In [4]:
data_full, df_attributes, _ = load_dataset_into_dataframes(n_cases=500)
datadic_full = create_datadict(data_full, df_attributes)

../imaterialist/data/datasets/make_dataset.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AttributesIds'][index] = row['AttributesIds'].split(',')
../imaterialist/data/datasets/make_dataset.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AttributesIds'][index] = [int(x) for x in df['AttributesIds'][index]]
../imaterialist/data/datasets/make_dataset.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AttributesId

In [5]:
n_train = 400
n_test = 100

datadic_train = datadic_full[:n_train].copy()
datadic_val = datadic_full[-n_test:].copy()

In [6]:
datadic_train.sample(10)

,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds,x0,y0,x1,y1
322,/home/julien/data-science/kaggle/imaterialist/...,1131858 13 1131917 36 1133630 42 1133718 40 11...,1800,1200,23,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",628,1418,692,1583
307,/home/julien/data-science/kaggle/imaterialist/...,212864 7 213881 21 214899 31 215920 35 216943 ...,1024,683,23,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",207,878,257,961
136,/home/julien/data-science/kaggle/imaterialist/...,179500 6 180515 16 181534 22 182557 25 183579 ...,1024,680,10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",175,163,530,956
370,/home/julien/data-science/kaggle/imaterialist/...,5669654 1 5673613 1 5677571 3 5681530 4 568548...,3960,2640,31,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1431,1546,2057,2923
288,/home/julien/data-science/kaggle/imaterialist/...,896338 1 897937 4 899537 7 901137 9 902736 13 ...,1600,1067,33,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",560,282,631,349
167,/home/julien/data-science/kaggle/imaterialist/...,247017 4 248399 11 249780 16 251162 18 252543 ...,1383,900,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",178,337,682,984
53,/home/julien/data-science/kaggle/imaterialist/...,701292 5 703597 15 705903 25 708209 34 710515 ...,2310,1536,6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",303,1181,824,2136
75,/home/julien/data-science/kaggle/imaterialist/...,1590945 1 1593942 4 1596939 7 1599938 8 160293...,3000,2000,37,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",530,929,741,1003
350,/home/julien/data-science/kaggle/imaterialist/...,243646 2 244218 6 244714 16 244743 3 244760 5 ...,576,1024,24,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",422,217,456,575
265,/home/julien/data-science/kaggle/imaterialist/...,1660869 5 1663151 15 1665436 21 1667723 22 167...,2287,1522,33,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",726,499,887,571


In [7]:

datadic_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ImageId        400 non-null    object
 1   EncodedPixels  400 non-null    object
 2   Height         400 non-null    int64 
 3   Width          400 non-null    int64 
 4   ClassId        400 non-null    int64 
 5   AttributesIds  400 non-null    object
 6   x0             400 non-null    int64 
 7   y0             400 non-null    int64 
 8   x1             400 non-null    int64 
 9   y1             400 non-null    int64 
dtypes: int64(7), object(3)
memory usage: 31.4+ KB


In [20]:
import pickle

In [10]:
datadict_train = pickle.load(open(path_data_interim / 'imaterialist_train_multihot_n=400.p', 'rb'))